### JOLTS Report

Updated October 6, 2020 with new JOLTS series IDs:
https://www.bls.gov/jlt/jlt_series_changes.htm

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

ind = {'000000': '\\textbf{Total nonfarm}',
       '110099': 'Mining \& logging',
       '230000': 'Construction',
       '320000': 'Durable goods manufacturing',
       '340000': 'Nondurable goods manufacturing',
       '510000': 'Information',
       '510099': 'Financial activities',
       '540099': 'Professional \& business services',
       '440000': 'Retail trade',
       '480099': 'Transportation, warehousing, \& utilities',
       '610000': 'Educational services',
       '620000': 'Health care \& social services',
       '720000': 'Accommodation \& food services',
       '910000': 'Federal government',
       '920000': 'State \& local government'}

In [2]:
series = {'JTS000000000000000JOL': 'Openings', 
          'JTS000000000000000HIL': 'Hires', 
          'JTS000000000000000QUL': 'Quits',
          'JTS000000000000000TSL': 'Separations',
          'JTS000000000000000LDL': 'Layoffs',
          'JTS000000000000000OSL': 'Other_Sep',
          'LNS13000000': 'Unemp'}

series = jolts_codes(series, 'QUR', ind)
    
df1 = bls_api(series, (2000, 2021), bls_key)

df1.to_csv(data_dir/'jolts_master.csv', index_label='date')

series2 = {}
series = jolts_codes(series2, 'JOR', ind)
    
df2 = bls_api(series2, (2000, 2021), bls_key)

df2.to_csv(data_dir/'jolts_master2.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED
Post Request Status: REQUEST_SUCCEEDED


In [2]:
df = (pd.read_csv(data_dir / 'jolts_master.csv', parse_dates=['date'])
        .set_index('date'))

levels_srs = ['Openings', 'Hires', 'Quits', 'Separations', 
              'Unemp', 'Layoffs', 'Other_Sep']
rates_srs = ['JTS720000000000000QUR', 'JTS000000000000000QUR']

levels_data = df[levels_srs].dropna() / 1000
levels_data.to_csv(data_dir / 'jolts.csv', index_label='date')

unjo = levels_data['Openings'] / levels_data['Unemp']
unjo.name = 'unjo'
unjo.to_csv(data_dir / 'unjo.csv', index_label='date')

ltdata = levels_data.iloc[-1]

ltdate = dtxt(ltdata.name)['mon1']
write_txt(text_dir / 'jolts_ltdate.txt', ltdate)
ltopen = ltdata['Openings']
pryropen = levels_data['Openings'].iloc[-13]
lthire = ltdata['Hires']
ltquit = ltdata['Quits']
ltsep = ltdata['Separations']
ltun = ltdata['Unemp']
ltlay = ltdata['Layoffs']
ltoth = ltdata['Other_Sep'] * 1_000_000
h19 = levels_data.loc['2019', 'Hires'].mean()
ts19 = levels_data.loc['2019', 'Separations'].mean()

text = (f'In {ltdate}, there were {ltopen:.1f} million total nonfarm '+
        'job openings (see {\color{blue!80!black}\\textbf{---}}) '
        f'and {lthire:.1f} million hires completed '+
        '(see {\color{cyan}\\textbf{---}}). In the same month, '+
        f'there were {ltsep:.1f} million nonfarm separations, '+
        f'including {ltlay:.1f} million layoffs '+
        '(see {\color{orange!70!yellow}\\textbf{---}}), '+
        f'{ltquit:.1f} million quits '+
        '(see {\color{red!85!violet}\\textbf{---}}), and '+
        f'{ltoth:,.0f} other separations. In 2019, there were an '
        f'average of {h19:.1f} million hires completed and '+
        f'{ts19:.1f} million total separations, per month.')
write_txt(text_dir / 'jolts2.txt', text)
print(text)

In December 2021, there were 10.9 million total nonfarm job openings (see {\color{blue!80!black}\textbf{---}}) and 6.3 million hires completed (see {\color{cyan}\textbf{---}}). In the same month, there were 5.9 million nonfarm separations, including 1.2 million layoffs (see {\color{orange!70!yellow}\textbf{---}}), 4.3 million quits (see {\color{red!85!violet}\textbf{---}}), and 392,000 other separations. In 2019, there were an average of 5.8 million hires completed and 5.7 million total separations, per month.


In [3]:
opun = levels_data['Openings'] / levels_data['Unemp']

ratio = opun.iloc[-1]
ratiop = opun.iloc[-2]
prmon = dtxt(opun.index[-2])['mon1']
ratio19 = opun.loc['2019'].mean()
color = 'violet'
write_txt(text_dir / 'unjo_node.txt', end_node(unjo, color, date='m'))
text = (f'In {ltdate}, there were {ltun:.1f} million unemployed '+
        f'people and {ltopen:.1f} million job openings, therefore '+
        'the ratio of job openings per unemployed person was '+
        f'{ratio:.1f} {c_line(color)}. In {prmon} the ratio was '+
        f'{ratiop:.1f}, and during 2019 the average ratio was '+
        f'{ratio19:.1f}.')
write_txt(text_dir / 'unjo.txt', text)
print(text)

In December 2021, there were 6.3 million unemployed people and 10.9 million job openings, therefore the ratio of job openings per unemployed person was 1.7 (see {\color{violet}\textbf{---}}). In November 2021 the ratio was 1.6, and during 2019 the average ratio was 1.2.


In [5]:
short_names = {'JTS000000000000000QUR': 'TOT_QU', 
               'JTS720000000000000QUR': 'AFS_QU'}
rates_data = df[rates_srs].dropna().rename(short_names, axis=1)

rates_data.to_csv(data_dir / 'quits.csv', index_label='date')

afs_col = 'red!50!purple'
node = end_node(rates_data['AFS_QU'], afs_col, percent=True)
write_txt(text_dir / 'quits_afs_node.txt', node)
tot_col = 'violet!90!black'
node = end_node(rates_data['TOT_QU'], tot_col, percent=True, 
                date='m')
write_txt(text_dir / 'quits_tot_node.txt', node)

ltdata = rates_data.iloc[-1]
afs_max = rates_data['AFS_QU'].max()
afs_idxmax = dtxt(rates_data['AFS_QU'].idxmax())['mon1']

h2 = ', the series high for the industry group.'
if afs_idxmax == ltdate:
    text2 = h2
else:
    text2 = ('; the series high for the industry group'+
            f' was {afs_max} percent in {afs_idxmax}.')

text = (f'In {ltdate}, the total quits rate in all industries '+
        f'was {ltdata.TOT_QU} percent {c_line(tot_col)}. The '+
        f'accommodations and food services quits rate was '+
        f'{ltdata.AFS_QU} percent {c_line(afs_col)}'+text2)
write_txt(text_dir / 'quits_afs.txt', text)
print(text)

In December 2021, the total quits rate in all industries was 2.9 percent (see {\color{violet!90!black}\textbf{---}}). The accommodations and food services quits rate was 6.1 percent (see {\color{red!50!purple}\textbf{---}}); the series high for the industry group was 6.6 percent in August 2021.


In [6]:
df = (pd.read_csv(data_dir / 'jolts_master.csv', parse_dates=['date'])
        .set_index('date'))

max_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().max()
min_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().min()
max_qur2 = df.loc['2018':,df.columns.str.endswith('QUR')].dropna().max()
min_qur2 = df.loc['2018':,df.columns.str.endswith('QUR')].dropna().min()
lt_qur = df.loc[:,df.columns.str.endswith('QUR')].dropna().iloc[-1]

qur = pd.DataFrame({'max': max_qur, 'min': min_qur, 'latest': lt_qur, 
                    'maxst': max_qur2, 'minst': min_qur2})

quits = {}
quits = jolts_codes(quits, 'QUR', ind, value='name')
    
final = pd.DataFrame()
final['outer1'] = qur['min']
final['range1'] = qur['minst'] - qur['min']
final['rangest'] = qur['maxst'] - qur['minst']
final['range2'] = qur['max'] - qur['maxst']
final['outer2'] = qur['max'].max() - qur['max']
final['latest'] = qur['latest']
final.index = final.index.map(quits)
final = final.sort_values('latest', ascending=False)
final['y'] = [i * -1 + 0.08 for i in list(range(0, len(qur)))]
final.index.name = 'name'
final.to_csv(data_dir / 'quits_ind.csv', sep=';')
#final

In [7]:
df = (pd.read_csv(data_dir / 'jolts_master2.csv', parse_dates=['date'])
        .set_index('date'))

max_jor = df.loc[:,df.columns.str.endswith('JOR')].dropna().max()
min_jor = df.loc[:,df.columns.str.endswith('JOR')].dropna().min()
max_jor2 = df.loc['2018':,df.columns.str.endswith('JOR')].dropna().max()
min_jor2 = df.loc['2018':,df.columns.str.endswith('JOR')].dropna().min()
lt_jor = df.loc[:,df.columns.str.endswith('JOR')].dropna().iloc[-1]

jor = pd.DataFrame({'max': max_jor, 'min': min_jor, 'latest': lt_jor, 
                    'maxst': max_jor2, 'minst': min_jor2})

jos = {}
jos = jolts_codes(jos, 'JOR', ind, value='name')
    
final = pd.DataFrame()
final['outer1'] = jor['min']
final['range1'] = jor['minst'] - jor['min']
final['rangest'] = jor['maxst'] - jor['minst']
final['range2'] = jor['max'] - jor['maxst']
final['outer2'] = jor['max'].max() - jor['max']
final['latest'] = jor['latest']
final.index = final.index.map(jos)
final = final.sort_values('latest', ascending=False)
final['y'] = [i * -1 + 0.08 for i in list(range(0, len(jor)))]
final.index.name = 'name'
final.to_csv(data_dir / 'openings_ind.csv', sep=';')
#final